In [ ]:
import pandas as pd
import os

In [ ]:
gsu_missingness_path = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/logs_01012023_233050/missingness.csv'
mimic_missingness_path = '/Users/jk1/temp/opsum_mimic/preprocessing/mimic_prepro_25012023_232713/logs_25012023_232713/missingness.csv'
var_to_english_map_path = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))),
                                                           'preprocessing/preprocessing_tools/feature_name_to_english_name_correspondence.xlsx')

In [ ]:
output_dir = '/Users/jk1/Downloads'

In [ ]:
gsu_missingness_df = pd.read_csv(gsu_missingness_path)
mimic_missingness_df = pd.read_csv(mimic_missingness_path)
var_to_english_map_df = pd.read_excel(var_to_english_map_path)

In [ ]:
gsu_total_n = gsu_missingness_df.n_missing_h1.max()
mimic_total_n = mimic_missingness_df.n_missing_h1.max()

In [ ]:
gsu_missingness_df['Geneva Dataset'] = gsu_missingness_df.n_missing_overall.apply(lambda x: f'{x} ({x / gsu_total_n * 100:.1f}%)')
mimic_missingness_df['MIMIC Dataset'] = mimic_missingness_df.n_missing_overall.apply(lambda x: f'{x} ({x / mimic_total_n * 100:.1f}%)')

In [ ]:
overall_df = gsu_missingness_df[['sample_label', 'Geneva Dataset']].merge(mimic_missingness_df[['sample_label', 'MIMIC Dataset']], on='sample_label', how='outer')

In [ ]:
overall_df

In [ ]:
# remove sample lables starting with max or min
overall_df = overall_df[~overall_df.sample_label.str.startswith('max')]
overall_df = overall_df[~overall_df.sample_label.str.startswith('min')]
# for samples starting with median, remove the median_ prefix
overall_df.loc[overall_df.sample_label.str.startswith('median'), 'sample_label'] = overall_df.loc[overall_df.sample_label.str.startswith('median'), 'sample_label'].apply(lambda x: x[7:])
# remove underscores in sample labels
overall_df.sample_label = overall_df.sample_label.apply(lambda x: x.replace('_', ' '))

In [ ]:
for feature in overall_df.sample_label.unique():
    if feature in var_to_english_map_df.feature_name.values:
        overall_df.loc[overall_df.sample_label == feature, 'sample_label'] = var_to_english_map_df[var_to_english_map_df.feature_name == feature].english_name.values[0]

In [ ]:
# capitalise first letter of each word
overall_df.sample_label = overall_df.sample_label.apply(lambda x: ' '.join([word.capitalize() for word in x.split(' ')]))

In [ ]:
# rename sample label column to Variable
overall_df.rename(columns={'sample_label': 'Variable'}, inplace=True)

In [ ]:
overall_df.to_csv(os.path.join(output_dir, 'missingness.csv'), index=False)